In [ ]:
import os,sys
sys.path.insert(0, os.path.abspath('..'))
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from pkldataset import PKLDataset

# Utility to set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Define model class
def get_model(input_length: int = 2800, num_classes: int = 10, input_channels: int = 1):
    class CNN(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Sequential(
                nn.Conv1d(input_channels, 16, kernel_size=31, padding=15),
                nn.BatchNorm1d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool1d(2)
            )
            self.conv2 = nn.Sequential(
                nn.Conv1d(16, 32, kernel_size=31, padding=15),
                nn.BatchNorm1d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool1d(2)
            )
            self.conv3 = nn.Sequential(
                nn.Conv1d(32, 64, kernel_size=31, padding=15),
                nn.BatchNorm1d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool1d(2)
            )
            conv_output_length = input_length // 8
            self.fc = nn.Sequential(
                nn.Flatten(),
                nn.Linear(64 * conv_output_length, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(128, num_classes)
            )

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            if x.dim() == 2:
                x = x.unsqueeze(1)
            x = self.conv1(x)
            x = self.conv2(x)
            x = self.conv3(x)
            return self.fc(x)
    return CNN()

# Training function
def train_model(model, loader, criterion, optimizer, scheduler,
                num_epochs, device, max_grad_norm=1.0):
    model.to(device)
    best_loss = float('inf')
    best_state = None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            y_idx = targets.argmax(dim=1)
            optimizer.zero_grad()
            out = model(inputs)
            loss = criterion(out, y_idx)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        scheduler.step()
        avg_loss = running_loss / len(loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

        # --- checkpointing logic ---
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

    # load best weights back into model before returning
    if best_state is not None:
        model.load_state_dict(best_state)

    return model

# Evaluation function
def eval_model(model, loader, device):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            y_idx = targets.argmax(dim=1)
            out = model(inputs)
            preds = out.argmax(dim=1)
            correct += (preds == y_idx).sum().item()
            total += targets.size(0)
    return 100. * correct / total

if __name__ == '__main__':
    # Configuration
    train_paths = ["../datasets/RPDC197/train_20", "../datasets/RPDC197/train_50", "../datasets/RPDC197/train_100", "../datasets/RPDC197/train_200", "../datasets/RPDC197/train_300",
 "../datasets/RPDC197/train_400", "../datasets/RPDC197/train_500", "../datasets/RPDC197/train_600"]

    # Validation datasets to test each model on
    val_paths = [
        "../datasets/RPDC185/val_1000",
        "../datasets/RPDC188/val_1000",
        "../datasets/RPDC191/val_1000",
        "../datasets/RPDC194/val_1000",
        "../datasets/RPDC197/val_1000",
    ]
    seeds = [101, 202, 303, 404, 505, 606, 707, 808, 909, 1001]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = nn.CrossEntropyLoss()

    # Container for results: {train_path: {val_path: [accs]}}
    results = {tp: {vp: [] for vp in val_paths} for tp in train_paths}

    # Multi-seed evaluation
    for seed in seeds:
        print(f"\n=== Seed {seed} ===")
        set_seed(seed)
        for tp in train_paths:
            print(f"-- Training on {tp}")
            # Prepare combined dataset
            ds_real = PKLDataset(tp)
            train_loader = DataLoader(ds_real, batch_size=32, shuffle=True)

            # Initialize and train model
            model = get_model().to(device)
            optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
            model = train_model(model, train_loader, criterion,
                                optimizer, scheduler, num_epochs=50,
                                device=device)

            # Evaluate on each validation set
            for vp in val_paths:
                val_loader = DataLoader(PKLDataset(vp), batch_size=64, shuffle=False)
                acc = eval_model(model, val_loader, device)
                results[tp][vp].append(acc)
                print(f"[{tp} -> {vp}] Seed {seed}: Acc = {acc:.2f}%")

    # Summary
    print("\n=== Summary across seeds ===")
    for tp in train_paths:
        for vp in val_paths:
            arr = np.array(results[tp][vp])
            mean, std = arr.mean(), arr.std(ddof=1)
            print(f"{tp} -> {vp}: Mean = {mean:.2f}%, Std = {std:.2f}%")



=== Seed 101 ===
-- Training on ../datasets/RPDC197/train_20
Epoch 1/50, Loss: 2.3675
Epoch 2/50, Loss: 3.1883
Epoch 3/50, Loss: 3.0981
Epoch 4/50, Loss: 2.4667
Epoch 5/50, Loss: 2.0693
Epoch 6/50, Loss: 1.8511
Epoch 7/50, Loss: 1.7181
Epoch 8/50, Loss: 1.4315
Epoch 9/50, Loss: 1.5203
Epoch 10/50, Loss: 1.0797
Epoch 11/50, Loss: 1.5396
Epoch 12/50, Loss: 1.2898
Epoch 13/50, Loss: 0.9009
Epoch 14/50, Loss: 0.9547
Epoch 15/50, Loss: 1.0292
Epoch 16/50, Loss: 0.6905
Epoch 17/50, Loss: 0.7343
Epoch 18/50, Loss: 0.8764
Epoch 19/50, Loss: 0.7880
Epoch 20/50, Loss: 1.2188
Epoch 21/50, Loss: 1.0892
Epoch 22/50, Loss: 0.8174
Epoch 23/50, Loss: 0.9089
Epoch 24/50, Loss: 1.0812
Epoch 25/50, Loss: 1.2072
Epoch 26/50, Loss: 0.7145
Epoch 27/50, Loss: 0.7652
Epoch 28/50, Loss: 0.9562
Epoch 29/50, Loss: 0.7491
Epoch 30/50, Loss: 0.8237
Epoch 31/50, Loss: 0.7510
Epoch 32/50, Loss: 0.9233
Epoch 33/50, Loss: 1.0441
Epoch 34/50, Loss: 1.0045
Epoch 35/50, Loss: 0.4035
Epoch 36/50, Loss: 0.9479
Epoch 37/50

In [4]:
# Build the summary dict keyed by the numeric RPDC ID
data = {}
for vp in val_paths:
    # extract e.g. 185 from "RPDC185/val_1000"
    key = int(vp.split('/')[-2].replace('RPDC', ''))
    means = []
    stds = []
    for tp in train_paths:
        arr = np.array(results[tp][vp])
        means.append(round(arr.mean(), 2))
        stds.append(round(arr.std(ddof=1), 2))
    data[key] = {'mean': means, 'std': stds}

In [6]:
data

{185: {'mean': [25.03, 48.52, 69.93, 69.34, 73.7, 70.16, 70.89, 72.39],
  'std': [4.6, 7.27, 2.21, 4.52, 5.35, 3.14, 3.97, 2.52]},
 188: {'mean': [15.92, 47.49, 69.12, 71.78, 78.84, 73.07, 74.6, 75.11],
  'std': [5.2, 11.42, 5.16, 8.89, 3.96, 4.34, 5.38, 6.33]},
 191: {'mean': [17.55, 56.55, 81.86, 89.31, 90.09, 90.54, 91.59, 90.13],
  'std': [7.16, 12.67, 6.77, 3.0, 3.99, 1.97, 1.42, 2.23]},
 194: {'mean': [22.05, 61.83, 85.19, 94.58, 95.47, 96.45, 98.07, 96.93],
  'std': [6.11, 14.22, 7.92, 4.33, 4.57, 1.86, 1.25, 1.64]},
 197: {'mean': [22.52, 74.22, 89.01, 96.12, 96.43, 97.76, 98.62, 98.5],
  'std': [5.67, 14.13, 6.54, 2.09, 2.82, 1.03, 0.5, 0.46]}}